# Part 3: Reverse Mode Automatic Differentiation with PyTorch

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

PyTorch implements Dynamic Reverse Mode Automatic Differentiation, much like we did in the previous exercise. There is one really major difference in what PyTorch provides over our simple example: it works directly with matrices (`Tensor`s) rather than with scalars (although obviously a matrix can represent a scalar).

In this tutorial, we'll explore PyTorch's AD implementation. Note that we're using the API of PyTorch 0.4 or later which simplifies use of AD (previous versions required wrapping all `Tensor`s that you wanted to compute gradients of in `Variable` objects; PyTorch 0.4 removes the need to do this and allows `Tensor`s themselves to track gradients).

We'll start with the simple example we tried earlier in the code block below:

__Task:__ Run the following code and verify the solution is correct

In [2]:
import torch

# set up the problem
x = torch.tensor(0.5, requires_grad=True)
y = torch.tensor(4.2, requires_grad=True)
z = x * y + torch.sin(x)

print("z = " + str(z.item()))

z.backward() # this goes through the computation graph and accumulates the gradients in the cached .grad attributes
print("dz/dx = " + str(x.grad.item()))
print("dz/dy = " + str(y.grad.item()))

z = 2.57942533493042
dz/dx = 5.077582359313965
dz/dy = 0.5


As with our own AD implementation, PyTorch lets us differentiate through an algorithm.

__Task__: Use the block below to compute the gradient $\partial z/\partial x$ of the following pseudocode algorithm and store the result in the `dzdx` variable:

    x = 0.5
    z = 1
    i = 0
    while i<2:
        z = (z + i) * x * x
        i = i + 1

In [3]:
x = torch.tensor(0.5, requires_grad=True)
z = torch.tensor(1.0, requires_grad=True)
for i in range(0,2):
    z = (z + i) * x * x

z.backward()
dzdx = x.grad.item()
print("dz/dx = {}".format(str(dzdx)))

dz/dx = 1.5


In [ ]:
assert dzdx

## PyTorch limitations: in-place operations and aliasing

PyTorch will throw an error at runtime if you try to differentiate through an in-place operation on a tensor. 

__Task__: Run the following code to see this in action.

In [5]:
x = torch.tensor(1.0, requires_grad=True)
y = x.tanh()
y.add_(3) # inplace addition
y.backward()

RuntimeError: ignored

Aliasing is also something that can't be differentiated through and will result in a slightly more cryptic error.

__Task__: Run the following code to see this in action. If you don't understand what this code does add some `print` statements to show the values of `x` and `y` at various points.

In [6]:
x = torch.tensor([1, 2, 3, 4], requires_grad=True, dtype=torch.float)
y = x[:1]
print(y)
y.add_(3)
print(y)
y.backward()

tensor([1.], grad_fn=<SliceBackward>)
tensor([4.], grad_fn=<AsStridedBackward>)


RuntimeError: ignored

## Dealing with multiple outputs

PyTorch can deal with the case where there are multiple output variables if we can formulate the expression in terms of tensor operations. Consider the example from the presentation for example:

$$\begin{cases}
     z = 2x + \sin x\\
     v = 4x + \cos x
\end{cases}$$

We could formulate this as:

$$
\begin{bmatrix}z \\ v\end{bmatrix} = \begin{bmatrix}2 \\ 4\end{bmatrix} \odot \bar{x} + \begin{bmatrix}1 \\ 0\end{bmatrix} \odot \sin\bar x + \begin{bmatrix}0 \\ 1\end{bmatrix} \odot \cos\bar x
$$

where 

$$
\bar x = \begin{bmatrix}x \\ x\end{bmatrix}
$$

and $\odot$ represents the Hadamard or element-wise product. This is demonstrated using PyTorch in the following code block.

__Task:__ run the code below.

In [7]:
x = torch.tensor([[1.0],[1.0]], requires_grad=True)

zv = ( torch.tensor([[2.0],[4.0]]) * x +
         torch.tensor([[1.0], [0.0]]) * torch.sin(x) +
         torch.tensor([[0.0], [1.0]]) * torch.cos(x) )
        
zv.backward(torch.tensor([[1.0],[1.0]])) # Note as we have "multiple outputs" we must pass in a tensor of weights of the correct size

print(x.grad)

tensor([[2.5403],
        [3.1585]])


__Task:__ Use the following box to write down the derivative of the expression for $\begin{bmatrix}z \\ v\end{bmatrix}$ and verify the gradients $\partial z/\partial x$ and $\partial v/\partial x$ are correct for $x=1$.

$ dz/dx = cos(x) + 2 = 2.5403, dv/dx = 4 - sin(x) = 3.1585 $ 

## Gradient descent & gradients with respect to a vector
Let's put everything together and using automatically computed gradients to find the minima of a function by taking steps down the gradient from an initial position. Rather than explicitly creating each input variable as a scalar as in the previous examples, we'll use a vector instead (so our gradients will be with respect to each element of the vector).

__Task:__ work through the following example to see how taking gradients with respect to a vector works & how simple gradient descent optimisation can be implemented.

In [8]:
# This is our starting vector
initial=[[2.0], [1.0], [10.0]]

# This is the function we will optimise (feel free to work out the analytic minima!)
def function(x):
    return x[0]**2 + x[1]**2 + x[2]**2

x = torch.tensor(initial, requires_grad=True, dtype=torch.float)
for i in range(0,100):
    # manually dispose of the gradient (in reality it would be better to detach and zero it to reuse memory)
    x.grad=None
    # evaluate the function
    J = function(x) 
    # auto-compute the gradients at the previously evaluated point x
    J.backward()
    # compute the update
    z = x - x.grad*0.1 
    x.data = z
    
    if i%10 == 0:
        print((x, function(x).item()))

(tensor([[1.6000],
        [0.8000],
        [8.0000]], requires_grad=True), 67.19999694824219)
(tensor([[0.1718],
        [0.0859],
        [0.8590]], requires_grad=True), 0.7747630476951599)
(tensor([[0.0184],
        [0.0092],
        [0.0922]], requires_grad=True), 0.008932411670684814)
(tensor([[0.0020],
        [0.0010],
        [0.0099]], requires_grad=True), 0.00010298370034433901)
(tensor([[0.0002],
        [0.0001],
        [0.0011]], requires_grad=True), 1.1873213452417986e-06)
(tensor([[2.2836e-05],
        [1.1418e-05],
        [1.1418e-04]], requires_grad=True), 1.3688886468798955e-08)
(tensor([[2.4520e-06],
        [1.2260e-06],
        [1.2260e-05]], requires_grad=True), 1.5782215811999123e-10)
(tensor([[2.6328e-07],
        [1.3164e-07],
        [1.3164e-06]], requires_grad=True), 1.8195657654207498e-12)
(tensor([[2.8270e-08],
        [1.4135e-08],
        [1.4135e-07]], requires_grad=True), 2.0978168543115717e-14)
(tensor([[3.0354e-09],
        [1.5177e-09],
        [

__Task__: Answer the following question in the box below: Why must the update in the code above be assigned to a different variable (the `z`) before being assigned to the `data` value of `x`?

Otherwise this might cause aliasing problems


## Differentiating through random operations

We'll end with an example that will be important later in the course: differentiating with respect to the parameters of a random number generator.

Assume that as some part of a differentiable program that we write we wish to incorporate a random element where we sample values, $z$ from a Normal distribution: $z \sim \mathcal{N}(\mu,\sigma^2)$. We want to learn the parameters of the generator $\mu$ and $\sigma^2$, but how can we do this? In a differentiable program setting we want to differentiate with respect to these parameters, but at first glance it isn't at all obvious what this means as the generator _just_ produces numbers which themselves don't have gradients.

The answer is often called the _reparameterisation trick_: If we note that sampling a Normal distribution is equivalent to drawing numbers from a Uniform distribution and scaling and shifting them: $z \sim \mathcal{N}(\mu,\sigma^2) \equiv z \sim \mu + \sigma\mathcal{U}(0,1)\equiv z = \mu + \sigma \zeta\, \rm{where}\, \zeta\sim\mathcal{U}(0,1)$. With this reparameterisation the gradients with respect to the parameters are obvious.

The following code block demonstrates this in practice; each of the gradients can be interpreted as how much an infintesimal change in $\mu$ or $\sigma$ would change $z$ if we could repeat the sampling operation again with the same value of `torch.rand(1)` being produced:

In [9]:
mu = torch.tensor(1.0, requires_grad=True)
sigma = torch.tensor(1.0, requires_grad=True)

for i in range(10):
    mu.grad = None
    sigma.grad = None
    
    z = mu + sigma * torch.rand(1)
    z.backward()
    print("z:", z.item(), "\tdzdmu:", mu.grad.item(), "\tdzdsigma:", sigma.grad.item())

z: 1.8982720375061035 	dzdmu: 1.0 	dzdsigma: 0.8982720375061035
z: 1.4771344661712646 	dzdmu: 1.0 	dzdsigma: 0.4771344065666199
z: 1.610734224319458 	dzdmu: 1.0 	dzdsigma: 0.6107342839241028
z: 1.9596350193023682 	dzdmu: 1.0 	dzdsigma: 0.9596349596977234
z: 1.8131669759750366 	dzdmu: 1.0 	dzdsigma: 0.8131669759750366
z: 1.7733500003814697 	dzdmu: 1.0 	dzdsigma: 0.7733500003814697
z: 1.7128384113311768 	dzdmu: 1.0 	dzdsigma: 0.712838351726532
z: 1.4010052680969238 	dzdmu: 1.0 	dzdsigma: 0.40100520849227905
z: 1.172426700592041 	dzdmu: 1.0 	dzdsigma: 0.1724267601966858
z: 1.269918441772461 	dzdmu: 1.0 	dzdsigma: 0.26991844177246094
